In [11]:
#importing libraries

import cv2
import os
import numpy as np
import pandas as pd
import pytesseract
from pytesseract import Output
import matplotlib.pyplot as plt
pytesseract.pytesseract.tesseract_cmd="/home/aman/anaconda3/envs/tallyInvoiceParser.env/bin/tesseract"
os.environ['TESSDATA_PREFIX'] = "/home/aman/anaconda3/envs/tallyInvoiceParser.env/share/tessdata"
import import_ipynb
from imagePreProcessor import *
from TesseractModify import *
from datetime import datetime
import math
from nltk.tag import StanfordNERTagger
os.environ['CLASSPATH'] = '/home/aman/Documents/Tally/Dependencies/StanfordNER/stanford-ner-2020-11-17/stanford-ner.jar'
os.environ['STANFORD_MODELS'] = '/home/aman/Documents/Tally/Dependencies/StanfordNER/stanford-corenlp-4.4.0-models-english/edu/stanford/nlp/models/ner/'
java_path = "/usr/lib/jvm/java-7-openjdk-amd64"
os.environ['JAVAHOME'] = java_path
stanford_classifier  =  '/home/aman/Documents/Tally/Dependencies/StanfordNER/stanford-corenlp-4.4.0-models-english/edu/stanford/nlp/models/ner/english.all.3class.caseless.distsim.crf.ser.gz'
from IPython.display import clear_output
from sklearn.cluster import KMeans
import difflib

In [2]:
st = StanfordNERTagger(stanford_classifier)

In [4]:
imagesFolderPath = '/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Data/Images'

In [5]:
def extractOrg(df):
    orgs = []
    texts = df["text"].to_numpy()
    T = ""
    for text in texts:
        T+=text+" "
    tagged = st.tag(str(T).split())
    for tag in tagged:
        if tag[1]=="ORGANIZATION" or tag[1]=="PERSON":
            orgs.append(tag[0])
    return orgs

In [6]:
def extract(df, orgs):
    indexes = []
    for index in df.index:
        if df["text"][index] not in orgs:
            indexes.append(index)
    D = df.drop(indexes)
    return D

In [8]:
def process(data, image):
    indexesToDrop = []
    stopwords = ["", " ", "  ", "/", ":", "-", ".", ",", "\n", "\t", "\\", "(", ")", "[", "]", "{", "}", "*", "&", "%", "$", "#", "|"]
    for index in data.index:
        if data["text"][index] in stopwords:
            indexesToDrop.append(index)   
    data.drop(indexesToDrop, inplace=True)

    data.reset_index(inplace=True)
    data.drop(columns=["index"], inplace=True)

    data["x"] = data["left"] + data["width"]/2
    data["y"] = data["top"] + data["height"]/2
    data["PageWidth"] = image.shape[1]
    data["PageHeight"] = image.shape[0]
    data["x"]/=data["PageWidth"]
    data["y"]/=data["PageHeight"]

    data["isTop"] = 0
    data["isBottom"] = 0
    data["isRight"] = 0
    data["isLeft"] = 0

    for index in data.index:
        data["text"][index] = data["text"][index].lower()

    neighbours = ["seller", "buyer", "receiver", "to", "from", "by", "customer", "purchaser", "supplier", "consignee", "client", "owner", "for", "store", "recipient"]

    neighboursX = ["seller x", "buyer x", "receiver x", "to x", "from x", "by x", "customer x", "purchaser x", "supplier x", "consignee x", "client x", "owner x", "for x", "store x", "recipient x"]
    
    neighboursY = ["seller y", "buyer y", "receiver y", "to y", "from y", "by y", "customer y", "purchaser y", "supplier y", "consignee y", "client y", "owner y", "for y", "store y", "recipient y"]

    for col in neighbours:
        data[col]=0
    for col in neighboursX:
        data[col]=0
    for col in neighboursY:
        data[col]=0
    
    

In [46]:
N = ["seller", "buyer", "receiver", "to", "from", "by", "customer", "purchaser", "supplier", "consignee", "client", "owner", "for", "store", "recipient"]
Neighbours = {"seller" : 1, "buyer" : -1, "receiver" : -1, "to" : -1, "from" : 1, "by" : 1, "customer" : -1, "purchaser" : -1, "supplier" : 1, "consignee" : -1, "client" : -1, "owner" : 1, "for" : -1, "store" : 1, "recipient" : -1}
neighboursX = ["seller x", "buyer x", "receiver x", "to x", "from x", "by x", "customer x", "purchaser x", "supplier x", "consignee x", "client x", "owner x", "for x", "store x", "recipient x"]
neighboursY = ["seller y", "buyer y", "receiver y", "to y", "from y", "by y", "customer y", "purchaser y", "supplier y", "consignee y", "client y", "owner y", "for y", "store y", "recipient y"]

In [13]:
def returnDist(x1, y1, x2, y2):
    p = [x1, y1]
    q = [x2, y2]
    return math.dist(p, q)

In [14]:
def compare(word, neighbours):
    if word in neighbours:
        return word
    else:
        matches = difflib.get_close_matches(word, neighbours, n=1, cutoff=0.8)
        if len(matches)==0:
            return None
        else:
            return matches[0]
            

In [60]:
def allocateNeighbours(df, N, neighbourFields, neighbourFieldsX, neighbourFieldsY):
    for index in df.index:
        x1 = df["x"][index]
        y1 = df["y"][index]
        block = df["block_num"][index]
        neighbours = {}
        distFact = 0.3
        for index2 in df.index:
            x2 = df["x"][index2]
            y2 = df["y"][index2]
            block2 = df["block_num"][index2]
            dist = returnDist(x1, y1, x2, y2)
            if x2<=x1 and y2<=y1:
                if dist<=distFact:
                    if df["text"][index2].lower() not in neighbours:
                        neighbours[df["text"][index2].lower()]=[x2, y2]
                    else:
                        if dist<=returnDist(x1, y1, neighbours[df["text"][index2].lower()][0], neighbours[df["text"][index2].lower()][1]):
                            neighbours[df["text"][index2].lower()]=[x2, y2]
                elif abs(block2-block)<=1:
                    if df["text"][index2].lower() not in neighbours:
                        neighbours[df["text"][index2].lower()]=[x2, y2]
                    else:
                        if dist<=returnDist(x1, y1, neighbours[df["text"][index2].lower()][0], neighbours[df["text"][index2].lower()][1]):
                            neighbours[df["text"][index2].lower()]=[x2, y2]
                    
        for i in range(len(N)):
            n = N[i]
            nx = neighbourFieldsX[i]
            ny = neighbourFieldsY[i]
            if compare(n, neighbours.keys())==None:
                pass
            else:
                df[n][index] = neighbourFields[n]
                coord = neighbours[compare(n, neighbours.keys())]
                df[nx][index]=coord[0]
                df[ny][index]=coord[1]

        df["NX"] = 0
        df["NY"] = 0

        for index in df.index:
            X = []
            Y = []
            for i in range(len(N)):
                n = df[N[i]][index]
                nx = df[neighbourFieldsX[i]][index]
                ny = df[neighbourFieldsY[i]][index]
                X.append(n*nx)
                Y.append(n*ny)
            df["NX"][index] = sum(X)
            df["NY"][index] = sum(Y)







In [61]:
def cluster(df):
    kmeans = KMeans(n_clusters=2, n_init="auto", random_state=0)
    X = df["x"].to_numpy()
    Y = df["y"].to_numpy()
    train = [[X[index], Y[index]] for index in range(len(X))]
    if len(train)<2:
        df["cluster"]=-1
        return
    kmeans.fit(train)
    df["cluster"]=0
    for index in df.index:
        x = df["x"][index]
        y = df["y"][index]
        pred = kmeans.predict([[x, y]])
        if pred==0:
            pass
        else:
            df["cluster"][index]=1

In [91]:
def merge(data):
    df = data.drop(columns=["level", "block_num", "page_num", "par_num", "line_num", "word_num", "left", "top", "width", "height"])
    conf = []
    T = ""
    x = []
    y = []
    NX = []
    NY = []
    for index in df.index:
        conf.append(df["conf"][index])
        T+=df["text"][index] + " "
        x.append(df["x"][index])
        y.append(df["y"][index])
        NX.append(df["NX"][index])
        NY.append(df["NY"][index])
        PW = df["PageWidth"][index]
        PH = df["PageHeight"][index]
        cluster = df["cluster"][index]
    dict = {}
    dictX = {}
    dictY = {}
    for col in N:
        if df[df[col]==1].shape[0]==0 and df[df[col]==-1].shape[0]==0:
            dict[col]=0
        elif df[df[col]==-1].shape[0]==0:
            dict[col]=max(df[df[col]>0][col].to_numpy())
        else:
            dict[col]=-1

    for col in neighboursX:
        dictX[col] = df[col].mean()

    for col in neighboursY:
        dictY[col] = df[col].mean()

    C = sum(conf)/len(conf)
    X = sum(x)/len(x)
    Y = sum(y)/len(y)
    NX = sum(NX)/len(NX)
    NY = sum(NY)/len(NY)
    merged = [C, T, X, Y, PW, PH, 0, 0, 0, 0]
    for col in N:
        merged.append(dict[col])
    for col in neighboursX:
        merged.append(dictX[col])
    for col in neighboursY:
        merged.append(dictY[col])
    merged.append(NX)
    merged.append(NY)
    merged.append(cluster)
    return merged
        
    

In [92]:
def concat(df):
    
    df.reset_index(inplace=True)
    df.drop(columns=["index"], inplace=True)
    
    if df[df["cluster"]==0].shape[0] == 0 :
        df.drop(columns=["level", "block_num", "page_num", "par_num", "line_num", "word_num", "left", "top", "width", "height"], inplace=True)
        return df
        
    df1 = df[df["cluster"]==0]
    df2 = df[df["cluster"]==1]
    rows = []
    row1 = merge(df1)
    row2 = merge(df2)
    rows.append(row1)
    rows.append(row2)
    for block in df1["block_num"].unique():
        df = df1[df1["block_num"]==block]
        r = merge(df)
        rows.append(r)
    for block in df2["block_num"].unique():
        df = df2[df2["block_num"]==block]
        r = merge(df)
        rows.append(r)
        
    df.drop(columns=["level", "block_num", "page_num", "par_num", "line_num", "word_num", "left", "top", "width", "height"], inplace=True)

    indexes = []
    for index in df.index:
        indexes.append(index)
    
    for row in rows:
        df.loc[len(df.index)] = row
        
    df.drop(indexes, inplace=True)
        
    return df
    
        
    
    

In [93]:
def create(imagePath):
    image = cv2.imread(imagePath)
    preProcessedImage = preProcessImage(imagePath)
    data = pytesseract.image_to_data(preProcessedImage, output_type=Output.DATAFRAME)
    data.dropna(inplace=True)
    data.reset_index(inplace=True)
    data.drop(columns=["index"], inplace=True)
    df = pd.DataFrame(data)
    process(df, image)
    allocateNeighbours(df, N, Neighbours, neighboursX, neighboursY)
    orgs = extractOrg(df)
    df = extract(df, orgs)
    cluster(df)
    d = concat(df)
    return d
    

In [94]:
def batchCreate(folderPath):
    os.chdir(folderPath)
    images = os.listdir()
    dfList = []
    for imagePath in images:
        #print(imagePath)
        df = create(imagePath)
        df["imageName"] = imagePath
        df["output"] = 0
        dfList.append(df)
    return dfList

In [95]:
os.chdir(imagesFolderPath)

In [96]:
imagePath = "28.jpeg"

In [97]:
df = create(imagePath)

/tmp/ipykernel_5831/2094090813.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["text"][index] = data["text"][index].lower()
/tmp/ipykernel_5831/2988146006.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["NX"][index] = sum(X)
/tmp/ipykernel_5831/2988146006.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["NY"][index] = sum(Y)
/tmp/ipykernel_5831/2988146006.py:34: SettingWithCopyWarning: 
A value is trying to be 

In [98]:
df

,conf,text,x,y,PageWidth,PageHeight,isTop,isBottom,isRight,isLeft,...,supplier y,consignee y,client y,owner y,for y,store y,recipient y,NX,NY,cluster
7,95.015106,tally solutions pvt. ltd. amr tech park ppc,0.438848,0.350246,2550,3300,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.262353,-0.244943,0


In [99]:
df.columns

Index(['conf', 'text', 'x', 'y', 'PageWidth', 'PageHeight', 'isTop',
       'isBottom', 'isRight', 'isLeft', 'seller', 'buyer', 'receiver', 'to',
       'from', 'by', 'customer', 'purchaser', 'supplier', 'consignee',
       'client', 'owner', 'for', 'store', 'recipient', 'seller x', 'buyer x',
       'receiver x', 'to x', 'from x', 'by x', 'customer x', 'purchaser x',
       'supplier x', 'consignee x', 'client x', 'owner x', 'for x', 'store x',
       'recipient x', 'seller y', 'buyer y', 'receiver y', 'to y', 'from y',
       'by y', 'customer y', 'purchaser y', 'supplier y', 'consignee y',
       'client y', 'owner y', 'for y', 'store y', 'recipient y', 'NX', 'NY',
       'cluster'],
      dtype='object')

In [101]:
df[df["cluster"]==0]["text"].to_numpy()

array(['tally solutions pvt. ltd. amr tech park ppc '], dtype=object)

In [102]:
dfList = batchCreate(imagesFolderPath)

/tmp/ipykernel_5831/2094090813.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["text"][index] = data["text"][index].lower()
/tmp/ipykernel_5831/2988146006.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["NX"][index] = sum(X)
/tmp/ipykernel_5831/2988146006.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["NY"][index] = sum(Y)
/tmp/ipykernel_5831/2988146006.py:34: SettingWithCopyWarning: 
A value is trying to be 

In [103]:
len(dfList)

110

In [118]:
resultDF = pd.concat(dfList)

In [119]:
resultDF.shape

(226, 60)

In [120]:
len(resultDF["imageName"].unique())

58

In [121]:
resultDF.drop(columns=["cluster"], inplace=True)

In [123]:
resultDF.reset_index(inplace=True)
resultDF.drop(columns=["index", "conf"], inplace=True)
resultDF.drop(columns=["level_0"], inplace=True)

In [124]:
resultDF.head()

,text,x,y,PageWidth,PageHeight,isTop,isBottom,isRight,isLeft,seller,...,consignee y,client y,owner y,for y,store y,recipient y,NX,NY,imageName,output
0,amr tech park amr tech park,0.222190,0.251364,2550,3300,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.120686,-0.213333,23.jpeg,0
1,il hongasandra ii hongasandra,0.325980,0.252652,2550,3300,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.120686,-0.213333,23.jpeg,0
2,amr tech park,0.222222,0.185758,2550,3300,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.116275,-0.148333,23.jpeg,0
3,amr tech park,0.222157,0.316970,2550,3300,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.125098,-0.278333,23.jpeg,0
4,il hongasandra,0.325980,0.188030,2550,3300,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.116275,-0.148333,23.jpeg,0


In [125]:
resultDF.describe()

,x,y,PageWidth,PageHeight,isTop,isBottom,isRight,isLeft,seller,buyer,...,supplier y,consignee y,client y,owner y,for y,store y,recipient y,NX,NY,output
count,226.000000,226.000000,226.000000,226.000000,226.0,226.0,226.0,226.0,226.000000,226.000000,...,226.000000,226.000000,226.000000,226.000000,226.000000,226.000000,226.000000,226.000000,226.000000,226.0
mean,0.396841,0.436372,2541.221239,3416.424779,0.0,0.0,0.0,0.0,0.053097,-0.035398,...,0.002560,0.003629,0.019863,0.008568,0.089920,0.003160,0.005361,-0.108371,-0.179480,0.0
std,0.230093,0.281525,514.382361,685.270254,0.0,0.0,0.0,0.0,0.224725,0.185194,...,0.015193,0.028881,0.093822,0.062001,0.204199,0.028092,0.044137,0.254674,0.368514,0.0
min,0.002711,0.021023,850.000000,1100.000000,0.0,0.0,0.0,0.0,0.000000,-1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.784879,-2.388030,0.0
25%,0.207373,0.221280,2479.000000,3300.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.160925,-0.278333,0.0
50%,0.352387,0.375554,2480.000000,3500.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
75%,0.577133,0.664420,2550.000000,3508.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
max,0.941835,0.975171,7747.000000,10959.000000,0.0,0.0,0.0,0.0,1.000000,0.000000,...,0.115154,0.276826,0.877576,0.631404,0.825399,0.337514,0.455224,0.945543,0.916334,0.0


In [126]:
for index in resultDF.index:
    name = resultDF["imageName"][index].split(".")[0]
    resultDF["imageName"][index]=int(name)

/tmp/ipykernel_5831/2442472026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultDF["imageName"][index]=int(name)


In [127]:
resultDF.sort_values("imageName", inplace=True)
resultDF

,text,x,y,PageWidth,PageHeight,isTop,isBottom,isRight,isLeft,seller,...,consignee y,client y,owner y,for y,store y,recipient y,NX,NY,imageName,output
154,private,0.198871,0.289954,2479,3504,0,0,0,0,0,...,0.276826,0.0,0.0,0.000000,0.0,0.0,-0.263211,-0.554509,2,0
153,cloud technologies private ltd.,0.667658,0.180151,2479,3504,0,0,0,0,0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,2,0
152,private cloud technologies private ltd.,0.784712,0.762842,2479,3504,0,0,0,0,0,...,0.000000,0.0,0.0,0.388898,0.0,0.0,-0.808552,-0.764155,2,0
151,cloud technologies private ltd. private privat...,0.534922,0.263780,2479,3504,0,0,0,0,0,...,0.079093,0.0,0.0,0.062806,0.0,0.0,-0.417046,-0.460107,2,0
39,renault,0.062856,0.307863,3508,2480,0,0,0,0,0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,pvt ltd tae pradesh,0.238240,0.106379,2753,3880,0,0,0,0,0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,108,0
128,pvt ltd tae pradesh,0.238240,0.106379,2753,3880,0,0,0,0,0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,108,0
131,mee coe,0.582637,0.055284,2753,3880,0,0,0,0,0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,108,0
129,mee coe,0.582637,0.055284,2753,3880,0,0,0,0,0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,108,0


In [128]:
pwd

'/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Data/Images'

In [129]:
os.chdir('/home/aman/Documents/Tally/Git-Document-AI/Document-AI/SellerNameModel/Dataset/')

In [130]:
pwd

'/home/aman/Documents/Tally/Git-Document-AI/Document-AI/SellerNameModel/Dataset'

In [131]:
resultDF.to_csv("dataset.csv")